# Retrieval Augmented Generation (RAG)

**Purpose**: Simple semantic search over a tiny corpus then feed result to LLM.

**Dependencies**: `openai`, `pandas`, `numpy`


In [ ]:
import numpy as np
import pandas as pd
from openai import OpenAI
client = OpenAI(base_url="http://0.0.0.0:11434/v1", api_key="ollama")

In [ ]:
docs = [
    'The earthquake caused major damage.',
    'Floods destroyed crops in the valley.',
    'Wildfires spread due to strong winds.'
]
df = pd.DataFrame({'text': docs})
df

In [ ]:
def embed(text):
    response = client.embeddings.create(model="mxbai-embed-large", input=text)
    return np.array(response.data[0].embedding)

In [ ]:
embeddings = np.vstack([embed(t) for t in df['text']])
embeddings

In [ ]:
query = 'What disaster damaged crops?'
q_vec = embed(query)
scores = embeddings @ q_vec
idx = scores.argmax()
context = df['text'][idx]
context

In [ ]:
def answer_with_context(question, context):
    prompt = f"{context}
Question: {question}
Answer:"
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(model="phi4-mini:latest", messages=messages)
    return {"response": response.choices[0].message.content.strip()}

In [ ]:
result = answer_with_context(query, context)
print(result['response'])